# Downloading XP spectra with astroquery.gaia


In [19]:
from astropy.table import vstack, hstack, Table
from astroquery.gaia import GaiaClass, Gaia
import numpy as np

g = GaiaClass(gaia_tap_server='https://gea.esac.esa.int/', gaia_data_server='https://gea.esac.esa.int/')
g.login(user = 'fanders')

INFO: Login to gaia TAP server [astroquery.gaia.core]
Password: ········
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK


## First query the xp_summary table 

In [2]:
query = "select top 2000 source_id from gaiadr3.xp_summary"
job     = g.launch_job_async(query, output_format = 'votable')
results = job.get_results()
results[0:5]
results.write("./data/test_xp_ids-result.fits", overwrite=True)

INFO: Query finished. [astroquery.utils.tap.core]


Or just load a list of source_ids of objects with XP spectra:

In [22]:
results = Table.read("./data/test_xp_ids-result.fits")

### Mingle a bit with the data to extract the interesting stuff from the datalink file

by defining some helper functions:

In [21]:
def chunks(lst, n):
    """
    Yield successive n-sized chunks from lst.
    """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
def extract_coefficient_table(datalink, n_coeffs_bp = 55, n_coeffs_rp = 55):
    """
    From a datalink file, extract the relevant info and create 
    a suitable table containing the BP/RP coefficients
    """
    t = datalink["XP_CONTINUOUS_COMBINED.xml"][0].to_table()
    
    names = tuple(["bp_coeff_"+str(ii) for ii in np.arange(n_coeffs_bp)])
    arr = np.zeros((len(t), len(names)), dtype=np.float32)
    for ii in np.arange(len(t)):
        arr[ii,:] = t["bp_coefficients"][ii]
    bp = Table(arr, names=names)

    names = tuple(["e_bp_coeff_"+str(ii) for ii in np.arange(n_coeffs_bp)])
    for ii in np.arange(len(t)):
        arr[ii,:] = t["bp_coefficient_errors"][ii]
    e_bp = Table(arr, names=names)

    names = tuple(["rp_coeff_"+str(ii) for ii in np.arange(n_coeffs_rp)])
    arr = np.zeros((len(t), len(names)), dtype=np.float32)
    for ii in np.arange(len(t)):
        arr[ii,:] = t["rp_coefficients"][ii]
    rp = Table(arr, names=names)

    names = tuple(["e_rp_coeff_"+str(ii) for ii in np.arange(n_coeffs_rp)])
    for ii in np.arange(len(t)):
        arr[ii,:] = t["rp_coefficient_errors"][ii]
    e_rp = Table(arr, names=names)

    s = hstack([t["source_id"], t["bp_n_relevant_bases"], t["rp_n_relevant_bases"], 
                bp, rp, e_bp, e_rp])
    
    return s

def extract_all_xp_info(datalink):
    """
    From a datalink file, extract the relevant info and create 
    a suitable table containing the BP/RP coefficients
    """
    t = datalink["XP_CONTINUOUS_COMBINED.xml"][0].to_table()
    return t

## Loop over chunks to download the data peu à peu

In [23]:
dl_threshold = 5000
ids          = results['source_id']
ids_chunks   = list(chunks(ids, dl_threshold))
datalink_all = []
ii0          = 0
print(f'* Input list contains {len(ids)} source_IDs')
print(f'* This list is split into {len(ids_chunks)} chunks of <= {dl_threshold} elements each')

retrieval_type = 'XP_CONTINUOUS'
data_structure = 'COMBINED'
data_release   = 'Gaia DR3'

for ii in np.arange(ii0, len(ids_chunks)):
    chunk_ii = ids_chunks[ii]
    print(f'Downloading Chunk #{ii+1}; N_files = {len(chunk_ii)}')
    datalink = g.load_data(ids=chunk_ii, data_release = data_release, retrieval_type=retrieval_type, 
                           format = 'votable', data_structure = data_structure)
    t = extract_all_xp_info(datalink)
    t.write("./data/xp_continuous_combined_chunk"+str(ii)+".vot", overwrite=True, format="votable")


* Input list contains 2000 source_IDs
* This list is split into 1 chunks of <= 5000 elements each


In [24]:
t

source_id,solution_id,bp_basis_function_id,bp_degrees_of_freedom,bp_n_parameters,bp_n_measurements,bp_n_rejected_measurements,bp_standard_deviation,bp_chi_squared,bp_coefficients,bp_coefficient_errors,bp_coefficient_correlations,bp_n_relevant_bases,bp_relative_shrinking,rp_basis_function_id,rp_degrees_of_freedom,rp_n_parameters,rp_n_measurements,rp_n_rejected_measurements,rp_standard_deviation,rp_chi_squared,rp_coefficients,rp_coefficient_errors,rp_coefficient_correlations,rp_n_relevant_bases,rp_relative_shrinking
int64,int64,int16,int16,int16,int16,int16,float32,float32,object,object,object,int16,float32,int16,int16,int16,int16,int16,float32,float32,object,object,object,int16,float32
4295806720,4545469030156206081,56,931,55,986,10,0.9934199,918.78815,[102.93398893929992 -12.336921213781045 -2.668856168170544\n -0.12631176306793765 -0.9347021092539146 0.05636787290132809\n -0.5756470580856333 -0.6846172385330567 0.059234816894635056\n 0.46950105196033415 -1.049452314512485 0.1609881785186126\n 0.2898638972289407 0.03374712423085008 -0.4063488099423453\n -1.3602904713293664 -0.3184426445853893 0.5882896688808781\n -0.3773160686104622 0.19038977235058613 0.06922277327980512\n -0.23448653316910048 -0.1722458715963882 -0.6142198686963983\n -0.13963998340839023 -0.1138812881896988 0.30074633840516274\n -0.12204213828319004 -0.7062276852727692 0.1443163782418436\n 0.09015936636508824 -0.33029444949063047 -0.07598260107968674\n 0.15860369820878784 -0.34716536706104456 -0.5839211326503775\n 0.16112634803338632 -0.35688616491028763 -0.042924270249539626\n 0.3017587834168448 0.12538800185351043 -0.5040434129231106\n -0.13388075517960601 -0.1604388462087038 -0.02560724947082449\n -0.18293295511319996 0.17704447097239784 0.09202146613696288\n 0.24382400729480166 0.00488172850257286 -0.09814218671433242\n 0.18724125907736267 0.040663635629979406 0.057853987652473945\n -0.011327088669908251],[0.5509409308433533 0.48301947116851807 0.49546295404434204\n 0.5173321962356567 0.5299262404441833 0.4601147770881653\n 0.4203845262527466 0.9257420301437378 0.5121223330497742\n 0.4571794271469116 0.5006343722343445 0.4566701352596283\n 0.4700485169887543 0.4014255702495575 0.40904927253723145\n 0.4547213315963745 0.4365222752094269 0.3828290104866028\n 0.4123351573944092 0.339138001203537 0.43870609998703003\n 0.38430002331733704 0.3937664330005646 0.38558217883110046\n 0.3617623448371887 0.3480507731437683 0.34165284037590027\n 0.3042541742324829 0.3185865879058838 0.2720094919204712\n 0.2973921298980713 0.33105990290641785 0.3145827651023865\n 0.2900508642196655 0.2818146347999573 0.28625044226646423\n 0.27657246589660645 0.28327682614326477 0.2550068497657776\n 0.27851298451423645 0.246046245098114 0.2526322901248932\n 0.2416275292634964 0.21144738793373108 0.1794297695159912\n 0.1686146855354309 0.17882096767425537 0.15285906195640564\n 0.12359446287155151 0.09689221531152725 0.08921550214290619\n 0.09339914470911026 0.06379669904708862 0.06738636642694473\n 0.026209484785795212],[0.11844608187675476 -0.1412540078163147 0.10548915714025497 ...\n -0.028390580788254738 -0.09018035978078842 0.10851791501045227],3,0.9996168,57,1109,55,1164,8,1.0217239,1157.7069,[113.55177710646585 -13.08779641421988 -0.16985882123205095\n 1.165527918643655 0.12352352307326303 -0.41699480018347135\n 0.4387806738191473 0.4829643969730511 1.0713035324121796\n 0.7549837695733127 -0.6165835516068882 -0.2832722885071944\n -1.123756458384809 -0.3564248448323243 -0.23375337137945307\n 0.11425840301892061 -0.48112233019704487 0.31644442464728456\n 0.12428814334576105 -0.3834589968724225 -0.8262083808932003\n 0.08770285842991365 -0.4880663690372998 -0.4750077181876673\n 0.35135375162000415 -0.34646144607819185 0.058804668685049896\n -0.1398513077515255 -0.20375304355674753 -0.2766411837599323\n 0.43945436679228544 0.5038078375376734 0.36213080347024806\n 0.4660511220626985 -0.21789530344956576 -0.15490454896962177\n -0.4069807097628909 -0.09212267536470867 -0.18275395351587784

## Download all sources explicitly mentioned in [Weiler+2023](https://ui.adsabs.harvard.edu/abs/2023A%26A...671A..52W/abstract)

In [34]:
source_ids     = [505171240162594560, 426306363477869696, 2067888218857234304, 5256686653442846336]

retrieval_type = 'XP_CONTINUOUS'
data_structure = 'COMBINED'
data_release   = 'Gaia DR3'

datalink = g.load_data(ids=source_ids, data_release = data_release, retrieval_type=retrieval_type, 
                       format = 'votable', data_structure = data_structure)
t = extract_all_xp_info(datalink)
t.write("./data/xp_continuous_combined_weiler2023_examples.vot", overwrite=True, format="votable")

In [37]:
t

source_id,solution_id,bp_basis_function_id,bp_degrees_of_freedom,bp_n_parameters,bp_n_measurements,bp_n_rejected_measurements,bp_standard_deviation,bp_chi_squared,bp_coefficients,bp_coefficient_errors,bp_coefficient_correlations,bp_n_relevant_bases,bp_relative_shrinking,rp_basis_function_id,rp_degrees_of_freedom,rp_n_parameters,rp_n_measurements,rp_n_rejected_measurements,rp_standard_deviation,rp_chi_squared,rp_coefficients,rp_coefficient_errors,rp_coefficient_correlations,rp_n_relevant_bases,rp_relative_shrinking
int64,int64,int16,int16,int16,int16,int16,float32,float32,object,object,object,int16,float32,int16,int16,int16,int16,int16,float32,float32,object,object,object,int16,float32
505171240162594560,4545469030156206080,56,1907,55,1962,20,1.0939294,2282.0715,[2811.092563650591 -796.0416959834034 162.5588668906897 -25.27826851150297\n 26.35361779643399 3.9886154792010675 15.754674419355652 -21.1363697778024\n 42.87889127844506 -2.1291276652192153 2.9135372695853046\n 0.14224140688410214 1.6902515917072471 -5.1525606908404775\n -0.48212449685768555 -5.830513275874344 0.3595665825868451\n -2.8766056950670365 -1.3052496982870772 3.528830284964264\n 1.4828130689250962 -1.455116921575788 -4.7298659586401435\n -4.454099641502483 0.14342694716256374 -1.1229568035067063\n -3.80901172898208 -5.782224321428516 0.1989721908032394\n -3.0725485237156676 -1.117040528941756 0.7864551657050374\n 1.2537525854571352 -0.7914580109135985 1.3084070447376117\n -2.240631878441327 -0.6178381388342559 -0.9910914362623002\n 0.19145132867747836 -0.4506879760680825 -0.2915280844033336\n 0.23497593039327946 0.2665407994164701 -0.5290191300753786\n -0.008420602004819144 -0.37630538629952504 -0.2416743304944261\n -0.5031919469561479 -0.6420945912975485 -0.4480589732653163\n 0.10709278439172076 -0.16438605080568394 -0.10253528232162215\n -0.055939377621126773 0.059887864600492426],[2.171191692352295 1.963774561882019 2.036078453063965 1.983081340789795\n 1.9997013807296753 1.659010887145996 1.5258839130401611 2.764164924621582\n 1.5709638595581055 1.733216404914856 1.8138262033462524\n 1.6636933088302612 1.730033040046692 1.2316726446151733\n 1.3278999328613281 1.6202622652053833 1.5583715438842773\n 1.2943779230117798 1.5225725173950195 1.1787841320037842\n 1.6158146858215332 1.499173879623413 1.4144037961959839\n 1.4175310134887695 1.3091832399368286 1.215772271156311 1.176416277885437\n 0.8142580389976501 1.0743446350097656 0.7318539023399353\n 0.9092584848403931 1.1452797651290894 0.7088251113891602\n 0.8692606687545776 0.6545555591583252 0.7099408507347107\n 0.6495113968849182 0.712705135345459 0.61503005027771 0.7406507134437561\n 0.5225951075553894 0.6846690773963928 0.8609294891357422\n 0.47254040837287903 0.539762020111084 0.7657744288444519\n 0.7715059518814087 0.5215336680412292 0.4494144320487976\n 0.31234779953956604 0.32411298155784607 0.16983044147491455\n 0.2469266802072525 0.12386519461870193 0.05271182209253311],[0.2216043472290039 -0.027897203341126442 0.1835343837738037 ...\n 0.02966872788965702 0.21224315464496613 0.058393508195877075],30,0.99999917,57,2855,55,2910,56,1.0875636,3376.8792,[2126.96863363693 -398.3925397750298 -95.89363702130582\n -24.699015047687816 2.6738141517678495 45.71455703477176\n 17.415390864420537 23.89394389414927 -3.406729194308485\n -4.063121436429951 -2.429823282658118 -33.39047527086101\n 0.0356333052690008 17.630725391648504 -20.197730292177127\n 2.4201327423752224 -2.4394085181142056 0.9133666495691706\n -6.904425711658882 5.986782238571689 -6.882708467051083 4.137747642083724\n 3.53984441241974 6.4213746225774795 2.146866528768938 3.2520009716959537\n -6.801375671390714 -2.5075184658642042 0.4071798663516245\n 2.9099723976013183 -4.715681888465579 2.3219887792008014\n 0.920248320543486 -2.9792001899384064 0.3113189257227959\n -1.3745050273535242 0.9797258659633276 -0.5717783221657661\n 0.29858127810078283 0.8569361626092308 -0.21822559065766262\n 0.15407783466060238 0.43810400403093147 -0.282298036890710